In [2]:
#библиотеки, необходимые для формирования запросов сайту и получение ответов
#и поиска в полученной с сайта информации необхоимых для проекта данных
from bs4 import BeautifulSoup
import requests as req
import lxml.html as html
import urllib3

import time #библиотека для установки паузы между запросами к сайту

#общие библиотеки
import pandas as pd
import numpy as np

import re

In [3]:
#формируем header для включения его в запросы, так как сайты могут блокировать запросы без него
header = {'user-agent': 'Chrome/108.0.0.0'}
urllib3.disable_warnings()

In [7]:
url_star = 'https://www.booking.com/searchresults.en-us.html?label=gen173nr-1FCAEoggI46AdIM1gEaMIBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuALNzcOcBsACAdICJGNhMWRiNzA1LWFmZDUtNGNmMC1iZmUyLWY1NGJhNTFhMTI5NNgCBeACAQ&aid=304142&ss=London%2C+Greater+London%2C+United+Kingdom&ssne=Milan&ssne_untouched=Milan&efdco=1&lang=en-us&src=index&dest_id=-2601889&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=4db987a60d13006f&ac_meta=GhA0ZGI5ODdhNjBkMTMwMDZmIAAoATICZW46BkxvbmRvbkAASgBQAA%3D%3D&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&nflt=ht_id%3D204%3Bclass%3D'
url_all_star = []

for i in range(2):
    url_all_star.append(url_star + str(i+4) + '&order=distance_from_search')

In [10]:
urls_hotel_4 = []
urls_hotel_5 = []

for star in range(2):
    time.sleep(3)
    if star == 0:
        for i in range(0, 376, 25):
            try:
                response = req.get(url_all_star[star] + '&offset=' + str(i), headers=header, timeout=50, verify=False)
                page = BeautifulSoup(response.text,'html.parser')
            except:
                print('Ошибка запроса')
                
                
            for a in page.find_all('a', class_='e13098a59f'):
                urls_hotel_4.append(a['href'])
        time.sleep(3) 
    if star == 1:
        for i in range(0, 151, 25):
            try:
                response = req.get(url_all_star[star] + '&offset=' + str(i), headers=header, timeout=50, verify=False)
                page = BeautifulSoup(response.text,'html.parser')
            except:
                print('Ошибка запроса')
                
                
            for a in page.find_all('a', class_='e13098a59f'):
                urls_hotel_5.append(a['href'])
        time.sleep(3)
    
    
        
print(len(urls_hotel_4))
print(len(urls_hotel_5))

399
167


In [11]:
urls_hotel_4_en = []
urls_hotel_5_en = []

    
for i in range(len(urls_hotel_4)):
    parts = urls_hotel_4[i].split('hapos=' + str(i+1))
    urls_hotel_4_en.append(parts[0] + ('hapos=' + str(i+1)) + '&lang=en-us' + parts[1])
    
for i in range(len(urls_hotel_5)):
    parts = urls_hotel_5[i].split('hapos=' + str(i+1))
    urls_hotel_5_en.append(parts[0] + ('hapos=' + str(i+1)) + '&lang=en-us' + parts[1])


print(len(urls_hotel_4_en))
print(len(urls_hotel_5_en))

399
167


In [12]:
data_array_full = np.empty(4, 'str')

tags = ['span', 'div', 'div']
classes = ['hp_address_subtitle js-hp_address_subtitle jq_tooltip', 'd8eab2cf7f c90c0a70d3 db63693c62',
           'b5cd09854e f0d4d6a2f5 e46e88563a']

all_star_url = [urls_hotel_4_en, urls_hotel_5_en]


count_star_0 = 3

for urls in all_star_url:
    count_star_0 += 1
    for url in urls:
        try:
            response = req.get(url, headers=header, timeout=50, verify=False)
            page = BeautifulSoup(response.text,'html.parser')
        except:
            print('ошибка запроса')
        
    
        data = []
        data.append(count_star_0)
        for i in range(3):
            try:
                elem = page.find_all(tags[i], classes[i])
                elem = elem[0].text
            except:
                elem = np.NaN
        
                
            data.append(elem)
            
        data_array = np.asarray(data).reshape(1,4)
        data_array_full = np.row_stack((data_array_full,data_array))
    
        time.sleep(3)

ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса


In [24]:
columns = np.array(['hotel_star', 'hotel_address','reviews_count','quality'])
hotel_london = pd.DataFrame(data=data_array_full, columns=columns)
hotel_london.replace('nan', np.NaN, inplace=True)
hotel_london = hotel_london.dropna(thresh=2).reset_index(drop=True)
hotel_london

,hotel_star,hotel_address,reviews_count,quality
0,,,,
1,4,"\n8 Northumberland Avenue, Westminster Borough...","4,830 reviews",Very Good
2,4,"\n8 Northumberland Avenue, Westminster Borough...","2,958 reviews",Very Good
3,4,"\n18-21 Northumberland Ave, Westminster Boroug...","2,927 reviews",Very Good
4,4,"\nThe Strand, Westminster Borough, London, WC2...","3,608 reviews",Excellent
...,...,...,...,...
562,5,"\n46 Westferry Circus , Tower Hamlets, London,...",623 reviews,Very Good
563,5,"\n22 Hertsmere Road, Canary Wharf, Tower Hamle...","1,997 reviews",Very Good
564,5,"\n22 Hertsmere Road, Canary Wharf, Tower Hamle...",60 reviews,Very Good
565,5,"\n1 Waterview Drive, Greenwich, London, SE10 0...","7,042 reviews",Excellent


In [25]:
hotel_df = pd.read_csv('C:/Users/Dmitriy/Desktop/SkillFactory/project_3/hotels.csv', sep=',')
hotel_df.shape

(386803, 57)

In [ ]:
dupl_columns = list(hotel_london.columns) 

mask = hotel_london.duplicated(subset=dupl_columns) 
hotel_duplicates = hotel_london[mask]
print(f'Число найденных дубликатов: {hotel_duplicates.shape[0]}')

In [ ]:
hotel_london = hotel_london.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {hotel_london.shape[0]}')

In [ ]:
hotel_london = hotel_london.drop_duplicates('hotel_address')

In [31]:
hotel_london['hotel_address'] = hotel_london['hotel_address'].apply(
    lambda x: re.sub("[^0-9a-zA-Z]"," ",x).lower().replace(' ', ''))

In [33]:
hotel_df['hotel_address'] = hotel_df['hotel_address'].apply(
    lambda x: x.lower().replace(' ', ''))

In [35]:
hotel_df = hotel_df.merge(
                hotel_london,
                on='hotel_address',
                how='left')

In [36]:
null = hotel_df[hotel_df['city_hotel'] == 'London'].isnull().mean() * 100
null_per = null[null > 0]
null_per

hotel_star       19.94979
reviews_count    20.03161
quality          20.03161
dtype: float64

In [39]:
hotel_df

,Unnamed: 0,hotel_address,additional_number_of_scoring,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,positive_review,review_total_positive_word_counts,...,city_hotel_2,city_hotel_3,city_hotel_4,city_hotel_5,city_hotel_6,coordinates_center,from_the_center,hotel_star,reviews_count,quality
0,0,strattonstreetmayfairwestminsterboroughlondonw...,581,8.4,The May Fair Hotel,United Kingdom,Leaving,3,Staff were amazing,4,...,0,0,0,0,0,"51.5085, -0.12574",1.247,5,"2,745 reviews",Very Good
1,1,130134southamptonrowcamdenlondonwc1b5afunitedk...,299,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,location,2,...,0,0,0,0,0,"51.5085, -0.12574",1.404,4,"1,178 reviews",Good
2,2,151bisruederennes6tharr75006parisfrance,32,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,No Positive,0,...,1,0,0,0,0,"48.8534, 2.3488",1.919,NaN,NaN,NaN
3,3,216avenuejeanjaures19tharr75019parisfrance,34,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,Friendly staff quiet comfortable room spotles...,11,...,1,0,0,0,0,"48.8534, 2.3488",5.164,NaN,NaN,NaN
4,4,molenwerf11014agamsterdamnetherlands,914,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,The staff was very friendly and helpful Break...,20,...,0,1,0,0,0,"52.374, 4.88969",3.177,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386798,386798,9knaresboroughplacekensingtonandchelsealondons...,107,9.0,Hotel Moonlight,France,No Negative,0,Tr s proche du metro Earl s court,10,...,0,0,0,0,0,"51.5085, -0.12574",4.812,NaN,NaN,NaN
386799,386799,landstraerhauptstrae15503landstrae1030viennaau...,272,8.4,BEST WESTERN PLUS Amedia Wien,Turkey,No Negative,0,The bed was so comfy I stayed with my boyfrie...,93,...,0,0,0,1,0,"48.2085, 16.3721",2.711,NaN,NaN,NaN
386800,386800,2931gowerstreetcamdenlondonwc1e6hgunitedkingdom,457,6.8,Bloomsbury Palace Hotel,Netherlands,room is really small but guess is normal in L...,12,great location simple check in out nice shower,9,...,0,0,0,0,0,"51.5085, -0.12574",1.417,4,"2,365 reviews",Pleasant
386801,386801,31greatcumberlandplacewestminsterboroughlondon...,365,8.1,The Marble Arch London,United Arab Emirates,No Negative,0,Location and very comfy bed,6,...,0,0,0,0,0,"51.5085, -0.12574",2.494,5,"1,713 reviews",Pleasant


In [38]:
hotel_london.to_csv('C:/Users/Dmitriy/Desktop/SkillFactory/project_3/parsing_hotels/hotel_london.csv')